In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
from Source.Classes import Model, epidemic
import itables.interactive
import ipywidgets as widgets
from IPython.display import display

# Define Generalized Logistic Growth Model

def GLM(t, r, p, K, C_0 = 1):
    def GLM_ode(C_t, t):
        return r*pow(C_t, p)*(1-C_t/K)
    GLM_int = odeint(GLM_ode, C_0, t)
    return GLM_int[:,0]

# Define default parameters for search

par0 = {"r": 0.8, "p": 1, "K":"0.2*pop"} # assumed total outbreak size is 20% of pop
parlower = {"r": 0, "p": 0, "K":"0.1*pop"} # assumed minimum outbreak size is 10% of pop
parupper = {"r": 10, "p": 1, "K":"0.8*pop"} # assumed max outbreak size is 80% of pop

# Initialise the Model

GLM_Model = Model("GLM", GLM, par0, parupper, parlower)

# Load and clean data

source = "https://raw.githubusercontent.com/tomwhite/covid-19-uk-data/master/data/covid-19-cases-uk.csv"
utla_raw = pd.read_csv(source) # Raw Upper Tier Local Authority (UTLA) Case Counts utla_raw.to_csv("dailycases.csv")
popdata = pd.read_csv("paramfit_cache.csv") # Load model fitted caches from 30/03/20

# Clean data

def str2num(s):
    
    '''
    float <- str
    
    Accounts for irregular phrases such as '1 to 4' in the counts data
    by taking the mean of all integers occuring in the phrase.
    '''
    
    assert type(s) is str, "Case counts is not a string"
    counts = [int(n) for n in s.split() if s.isdigit()]
    if len(counts):
        return np.mean(counts)
    return np.nan
        
utla_raw.TotalCases = [str2num(s) for s in utla_raw.TotalCases]
utla_raw.dropna(subset=['TotalCases'],inplace=True)
utla_raw.Date = pd.to_datetime(utla_raw.Date)
utla_raw = utla_raw[utla_raw.Date < pd.to_datetime('today').strftime("%m/%d/%Y")] # remove today's data as it seems to be underreported

# Load the data into the Epidemic class, register that cache is being used

UKcases = epidemic(casedata = utla_raw, popdata = popdata)
UKcases.all_fitted = True
UKcases.all_fitted_bootstrap = True

<IPython.core.display.Javascript object>

In [2]:
# Visualise and explore case counts and projections per city, where sufficient data available

city_dropdown = widgets.Dropdown(options = np.sort(UKcases.popdata.Name.values), description="Choose target city: ")
intervals_dropdown = widgets.Dropdown(options = [False, True], description="Include error bars: ")

graph_output = widgets.Output()

def and_filter(city, intervals):
    graph_output.clear_output()
    with graph_output:
        UKcases.fit_target(city, GLM_Model, visualise=True, assess=False, intervals=intervals)

def city_dropdown_eventhandler(change):
    and_filter(change.new, intervals_dropdown.value)

def intervals_dropdown_eventhandler(change):
    and_filter(city_dropdown.value, change.new)
    
city_dropdown.observe(city_dropdown_eventhandler, names='value')
intervals_dropdown.observe(intervals_dropdown_eventhandler, names='value')

display(city_dropdown)
display(intervals_dropdown)

display(graph_output)

Dropdown(description='Choose target city: ', options=('Barking and Dagenham', 'Barnet', 'Barnsley', 'Bath and …

Dropdown(description='Include error bars: ', options=(False, True), value=False)

Output()

In [3]:
# View table of fitted parameters (and standard errors)

UKcases.popdata.sort_values(by=['r', 'p'],ascending=False)

Unnamed: 0       Code                         Name               Geog  \
0           137  E06000020           Telford and Wrekin  Unitary Authority   
1           380  W06000018                   Caerphilly  Unitary Authority   
2           226  E09000020       Kensington and Chelsea     London Borough   
3           235  E09000003                       Barnet     London Borough   
4           373  W06000011                      Swansea  Unitary Authority   
..          ...        ...                          ...                ...   
151         371  W06000009                Pembrokeshire  Unitary Authority   
152         172  E06000031                 Peterborough  Unitary Authority   
153          66  E06000013           North Lincolnshire  Unitary Authority   
154         256  E06000046                Isle of Wight  Unitary Authority   
155          64  E06000010  Kingston upon Hull, City of  Unitary Authority   

       Area  Population  PopDensity  MedAge         r             p  \
0     290.0    177799.0       612.0    39.4  2.111644  3.064196e-01   
1     277.0    181019.0       653.0    41.6  1.894286  1.213053e-01   
2      12.0    156197.0     12884.0    39.4  1.688543  3.474143e-01   
3      87.0    392140.0      4520.0    37.0  1.544515  3.543301e-01   
4     380.0    246466.0       649.0    39.6  1.542405  3.516786e-01   
..      ...         ...         ...     ...       ...           ...   
151  1619.0    125055.0        77.0    48.0  0.132886  9.306109e-11   
152   343.0    201041.0       585.0    36.5  0.131098  1.000000e+00   
153   846.0    172005.0       203.0    44.1  0.130766  1.000000e+00   
154   380.0    141538.0       373.0    50.1  0.097806  1.000000e+00   
155    71.0    260645.0      3648.0    35.6  0.080775  1.000000e+00   

                 K      r_se      p_se           K_se  LatestTotalCases  
0    142239.199980  1.014728  0.209755   63462.025342              40.0  
1     18101.900001  0.851043  0.241034   47380.359104              23.0  
2    124957.600000  0.557368  0.081189   44629.013289             170.0  
3     39214.000000  0.446951  0.068896  139061.670498             124.0  
4     24646.600000  0.905777  0.283197   86262.999624              19.0  
..             ...       ...       ...            ...               ...  
151   12505.500011  0.078037  0.414301   37546.456791               2.0  
152  160832.799984  0.036571  0.113226   68934.477410              24.0  
153  137604.000000  0.091008  0.295068   55175.809214              15.0  
154  113230.400000  0.076047  0.341268   46283.782362              11.0  
155  208515.999998  0.077673  0.431404   87323.207144              11.0  

[156 rows x 15 columns]

In [4]:
# Plot with confidence intervals

fig = UKcases.popdata.iplot(x="r",y="p",mode="markers",
                            size=np.log(UKcases.popdata.LatestTotalCases)*3,
                            xTitle="Growth rate (r)", yTitle="Growth rate dampening (p)",
                            title="Growth rate vs growth rate dampening per city in England + Wales",
                            asFigure=True)

fig.update_yaxes(autorange="reversed")

r_se_up = np.where(UKcases.popdata.r.values+UKcases.popdata.r_se.values>10, 10-UKcases.popdata.r.values, UKcases.popdata.r_se.values)
r_se_down = np.where(UKcases.popdata.r.values-UKcases.popdata.r_se.values<0, UKcases.popdata.r.values, UKcases.popdata.r_se.values)

p_se_up = np.where(UKcases.popdata.p.values+UKcases.popdata.p_se.values>1, 1-UKcases.popdata.p.values, UKcases.popdata.p_se.values)
p_se_down = np.where(UKcases.popdata.p.values-UKcases.popdata.p_se.values<0, UKcases.popdata.p.values, UKcases.popdata.p_se.values)

fig.update_traces(hovertemplate = '<br><b>%{text}</b>'
                                  '<br>Growth rate (r): %{x:.2f}'
                                  '<br>Growth dampening (p): %{y:.2f}',
                  text = [x+" (latest total cases: "+str(int(y))+")" for (x,y) in zip(UKcases.popdata.Name.values, UKcases.popdata.LatestTotalCases.values)],
                  error_y=dict(
                            type='data', # value of error bar given in data coordinates
                            symmetric=False,
                            array = p_se_up,
                            arrayminus = p_se_down,
                            color='rgba(147,112,219,0.1)',
                            visible=True),
                  error_x=dict(
                            type='data', # value of error bar given in data coordinates
                            symmetric=False,
                            array=r_se_up,
                            arrayminus=r_se_down,
                            color='rgba(147,112,219,0.1)',
                            visible=True),
                                )

fig.show()